## <b>4.3 PREPROCESSAMENT I ANÀLISI DE DADES</b>

### <b>4.3.5 Divisió del conjunt de dades</b>

In [3]:
# ============================================================
# 4.3.5 DIVISIÓ DEL CONJUNT DE DADES
# ============================================================
# Objectiu:
#   - Carregar els datasets "full" generats a 4.3.4.6 després
#     de l'enginyeria de variables.
#   - Aplicar una partició temporal coherent per a TOTS els models:
#         * Train: anys 2015–2017
#         * Test : any 2018
#   - Generar versions train/test per a:
#         * Model de freqüència – GLM/GAM
#         * Model de freqüència – GBM/GBDT
#         * Model de severitat
#         * (Opcional) Model de rendibilitat (ràtio)
#   - Deixar els fitxers desats a data/model per ser consumits
#     directament al capítol 4.4 (modelatge).
#
# Notes metodològiques:
#   - La divisió és EXCLUSIVAMENT temporal (Policy_year),
#     no aleatòria → evita "data leakage" de futur.
#   - La variable set_type ja existeix com a metadada, però
#     aquí el tall es fa explícitament per Policy_year.
#   - La validació interna dels models es farà amb k-fold
#     sobre el conjunt de train; aquí no es crea un "validation"
#     separat.
# ============================================================

import os
import pandas as pd

# Ajustos de visualització per a inspeccions a consola (Annex)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# ------------------------------------------------------------
# 0) Rutes dels datasets "full" generats a 4.3.4.6
#    (freqüència GLM, freqüència GBM, severitat i ràtio econòmica)
# ------------------------------------------------------------

# Directori base on s'han desat els datasets finals després de l'enginyeria
base_model_path = "data/model"

# Rutes individuals de cada dataset
freq_glm_full_path  = os.path.join(base_model_path, "freq_glm_full.csv")   # freqüència GLM/GAM (ja reduït)
freq_gbm_full_path  = os.path.join(base_model_path, "freq_gbm_full.csv")   # freqüència GBM/GBDT (feature set complet)
sev_full_path       = os.path.join(base_model_path, "sev_full.csv")        # severitat (només sinistres amb cost)
ratio_full_path     = os.path.join(base_model_path, "ratio_full.csv")      # (opcional) rendibilitat / ràtio econòmica

# ------------------------------------------------------------
# 1) Càrrega dels datasets "full"
# ------------------------------------------------------------

# Carrega els tres datasets principals (freq GLM, freq GBM i severitat)
df_freq_glm_full = pd.read_csv(freq_glm_full_path)
df_freq_gbm_full = pd.read_csv(freq_gbm_full_path)
df_sev_full      = pd.read_csv(sev_full_path)

# El dataset de ràtio és opcional (pot no existir)
if os.path.exists(ratio_full_path):
    df_ratio_full = pd.read_csv(ratio_full_path)
else:
    df_ratio_full = None

print("Dimensions carregades des de data/model:")
print("  Freq GLM full:", df_freq_glm_full.shape)
print("  Freq GBM full:", df_freq_gbm_full.shape)
print("  Severitat full:", df_sev_full.shape)
if df_ratio_full is not None:
    print("  Ràtio full:", df_ratio_full.shape)
else:
    print("  Ràtio full: no disponible (fitxer inexistent)")

# ------------------------------------------------------------
# 1.1) Assegurar que la columna temporal té el tipus correcte
#      (per robustesa, en cas que algun CSV s'hagi desat amb strings)
# ------------------------------------------------------------

YEAR_COL = "Policy_year"

for name, df in [
    ("Freq GLM full", df_freq_glm_full),
    ("Freq GBM full", df_freq_gbm_full),
    ("Severitat full", df_sev_full),
]:
    if YEAR_COL not in df.columns:
        raise ValueError(f"La columna {YEAR_COL} no existeix a {name}.")
    # Convertim a enter (si ja ho és, no passa res)
    df[YEAR_COL] = df[YEAR_COL].astype(int)

# Per al dataset opcional de ràtio
if df_ratio_full is not None:
    if YEAR_COL not in df_ratio_full.columns:
        raise ValueError(f"La columna {YEAR_COL} no existeix al dataset de ràtio.")
    df_ratio_full[YEAR_COL] = df_ratio_full[YEAR_COL].astype(int)

# ------------------------------------------------------------
# 2) Definició de la partició temporal
# ------------------------------------------------------------

# Anys que entren al conjunt de training
TRAIN_YEARS = [2015, 2016, 2017]

# Any reservat com a conjunt de test (futur)
TEST_YEAR = 2018

def temporal_train_test_split(df,
                              year_col: str = YEAR_COL,
                              train_years = TRAIN_YEARS,
                              test_year: int = TEST_YEAR):
    """
    Aplica una divisió temporal train/test sobre el dataframe d'entrada.

    Parameters
    ----------
    df : pd.DataFrame
        Dataset complet sobre el qual es vol aplicar l'split.
    year_col : str
        Nom de la columna que conté l'any de pòlissa (Policy_year).
    train_years : list[int]
        Llista d'anys que es consideraran training.
    test_year : int
        Any que es considerarà test (futur).

    Returns
    -------
    train : pd.DataFrame
        Subset amb registres dels anys train_years.
    test : pd.DataFrame
        Subset amb registres de l'any test_year.

    Notes
    -----
    - Aquesta funció NO barreja anys ni fa sampling aleatori:
      és una divisió purament temporal.
    - És aplicable tant a freqüència com a severitat i ràtio,
      sempre que comparteixin la columna year_col.
    """
    if year_col not in df.columns:
        raise ValueError(f"La columna temporal {year_col} no existeix al dataset.")

    # Subset per als anys de training (poden ser múltiples)
    train = df[df[year_col].isin(train_years)].copy()

    # Subset per a l'any de test (normalment 1 any: el més recent)
    test  = df[df[year_col] == test_year].copy()

    # Comprovació bàsica de que no hi ha superposició d'anys
    years_train = set(train[year_col].unique())
    years_test  = set(test[year_col].unique())
    intersection = years_train.intersection(years_test)
    if intersection:
        raise ValueError(f"Hi ha anys solapats entre train i test: {intersection}")

    return train, test

# ------------------------------------------------------------
# 3) Aplicar la divisió temporal a tots els datasets
# ------------------------------------------------------------

# 3.1) Freqüència GLM (dataset ja reduït de 4.3.4.6)
freq_glm_train, freq_glm_test = temporal_train_test_split(df_freq_glm_full)
print("\n=== Freqüència GLM ===")
print("Train shape :", freq_glm_train.shape)
print("Test shape  :", freq_glm_test.shape)
print("Train anys:", freq_glm_train[YEAR_COL].value_counts().to_dict())
print("Test anys :", freq_glm_test[YEAR_COL].value_counts().to_dict())

# 3.2) Freqüència GBM (feature set complet)
freq_gbm_train, freq_gbm_test = temporal_train_test_split(df_freq_gbm_full)
print("\n=== Freqüència GBM ===")
print("Train shape :", freq_gbm_train.shape)
print("Test shape  :", freq_gbm_test.shape)
print("Train anys:", freq_gbm_train[YEAR_COL].value_counts().to_dict())
print("Test anys :", freq_gbm_test[YEAR_COL].value_counts().to_dict())

# 3.3) Severitat (només pòlisses amb sinistre i cost>0)
sev_train, sev_test = temporal_train_test_split(df_sev_full)
print("\n=== Severitat ===")
print("Train shape :", sev_train.shape)
print("Test shape  :", sev_test.shape)
print("Train anys:", sev_train[YEAR_COL].value_counts().to_dict())
print("Test anys :", sev_test[YEAR_COL].value_counts().to_dict())

# 3.4) Ràtio econòmica (opcional)
if df_ratio_full is not None:
    ratio_train, ratio_test = temporal_train_test_split(df_ratio_full)
    print("\n=== Rendibilitat (Ràtio) ===")
    print("Train shape :", ratio_train.shape)
    print("Test shape  :", ratio_test.shape)
    print("Train anys:", ratio_train[YEAR_COL].value_counts().to_dict())
    print("Test anys :", ratio_test[YEAR_COL].value_counts().to_dict())
else:
    ratio_train = ratio_test = None

# ------------------------------------------------------------
# 3.5) Comprovacions de consistència (opcionales però recomanables)
#      Assegurem que la suma de files train+test coincideix amb el full
# ------------------------------------------------------------

assert len(df_freq_glm_full) == len(freq_glm_train) + len(freq_glm_test), \
    "Mismatch en freq GLM (train+test != full)"
assert len(df_freq_gbm_full) == len(freq_gbm_train) + len(freq_gbm_test), \
    "Mismatch en freq GBM (train+test != full)"
assert len(df_sev_full) == len(sev_train) + len(sev_test), \
    "Mismatch en severitat (train+test != full)"
if df_ratio_full is not None:
    assert len(df_ratio_full) == len(ratio_train) + len(ratio_test), \
        "Mismatch en ràtio (train+test != full)"

print("\n✔ Comprovacions de consistència superades (train + test = full).")

# ------------------------------------------------------------
# 4) Desar datasets train/test per al modelatge (capítol 4.4)
# ------------------------------------------------------------

# Ens assegurem que el directori existeix (tot i que ja hauria d'existir)
os.makedirs(base_model_path, exist_ok=True)

# 4.1) Freqüència GLM
freq_glm_train.to_csv(os.path.join(base_model_path, "freq_glm_train.csv"), index=False)
freq_glm_test.to_csv(os.path.join(base_model_path, "freq_glm_test.csv"), index=False)

# 4.2) Freqüència GBM
freq_gbm_train.to_csv(os.path.join(base_model_path, "freq_gbm_train.csv"), index=False)
freq_gbm_test.to_csv(os.path.join(base_model_path, "freq_gbm_test.csv"), index=False)

# 4.3) Severitat
sev_train.to_csv(os.path.join(base_model_path, "sev_train.csv"), index=False)
sev_test.to_csv(os.path.join(base_model_path, "sev_test.csv"), index=False)

# 4.4) Ràtio econòmica (si existeix)
if ratio_train is not None:
    ratio_train.to_csv(os.path.join(base_model_path, "ratio_train.csv"), index=False)
    ratio_test.to_csv(os.path.join(base_model_path, "ratio_test.csv"), index=False)

print("\n4.3.5 complet – Datasets train/test temporals generats i desats a data/model/")
print("   (La validació interna dels models es farà via k-fold sobre el conjunt de train al capítol 4.4.)")


Dimensions carregades des de data/model:
  Freq GLM full: (105555, 41)
  Freq GBM full: (105555, 47)
  Severitat full: (19646, 38)
  Ràtio full: (105555, 26)

=== Freqüència GLM ===
Train shape : (69740, 41)
Test shape  : (35815, 41)
Train anys: {2017: 33753, 2016: 31428, 2015: 4559}
Test anys : {2018: 35815}

=== Freqüència GBM ===
Train shape : (69740, 47)
Test shape  : (35815, 47)
Train anys: {2017: 33753, 2016: 31428, 2015: 4559}
Test anys : {2018: 35815}

=== Severitat ===
Train shape : (16259, 38)
Test shape  : (3387, 38)
Train anys: {2016: 8888, 2017: 6004, 2015: 1367}
Test anys : {2018: 3387}

=== Rendibilitat (Ràtio) ===
Train shape : (69740, 26)
Test shape  : (35815, 26)
Train anys: {2017: 33753, 2016: 31428, 2015: 4559}
Test anys : {2018: 35815}

✔ Comprovacions de consistència superades (train + test = full).

4.3.5 complet – Datasets train/test temporals generats i desats a data/model/
   (La validació interna dels models es farà via k-fold sobre el conjunt de train al cap